 # Sumário
 
###### - <a id='#8'>  </a>  

- <a href='#1'>1. Introdução</a>  
- <a href='#2'>2. Aplicando cada fórmula nas portas</a>  
- <a href='#3'>3. As classes  TruthsGenerates, são responsáveis por gerar todas as combinações binárias da tabela verdade.</a>  
- <a href='#4'>4. Essa é a etapa de gerar a tabela verdade do Circuito com falha ou sem falha </a>  

    - <a href='#4.1'>4.1 Leitura dos csvs gerados. Com falha, sem falha e os csvs das saídas dos circuitos. </a>  

- <a href='#5'>5 Comparando as falhas com o circuito de erro e observando quais falhas são iguais as saidas do circuito</a>  

    - <a href='#5.1'>5.1 Resultado do diagnóstico</a>  

- <a href='#6'>6. Abaixo salvamos as saídas do circuito, fazemos um diagnóstico</a>  

- <a href='#7'> 7. Tempo de Execução do Diagnostico</a>  



# - <a id='#1'>1. Introdução</a> 
###### - <a href='#8'> voltar ao sumário </a>  


In [872]:
from prettytable import PrettyTable, from_csv
from itertools import permutations
import pandas as pd
import numpy as np
import itertools
from random import randint
from itertools import chain
import string
import numpy
import lxml
import time
import csv
import re


In [873]:
     #CPU
time_init = time.time()
time_diag = time.time()


Dependencias: jupyter notebook, python 3.7

##### Cada variável de entrada do circuito será indicado por letras do alfabeto.
##### Cada porta do circuito será indicado letra letra p e o número da porta. Exemplo p0,p1...pn.

#### Descrição do circuito lógico. var indica as variáveis de entrada, coloque as variáveis de entrada em uma lista separado por vírgula e sem espaço.
Para representar as Falhas de entrada coloque E0 para representar uma entrada presa em 0 e E1 para representar uma entrada presa em 1

###### p0 = [ A = 0 ] indica que p0 está presa em 0 para a Entrada no A, caso a porta não possua falhas de entrada coloque uma lista vazia mesmo assim, por exemplo p1 = [  ]  indica que p1 não tem falhas. Lembrando que é necessário colocar espaço em cada caractere.

Fout representa falhas de saída para as portas, no caso só coloque na lista as postas que tem falha e o valor, no exemplo temos p1=0 que
indica que p1 está preso na saída da porta por 0.

###### Em seguida temos a representação das portas com sua entrada lógica. Por exemplo, p0 = ( A and B ). É necessário coloca colocar espaço em cada caractere de entrada.


######  var = [ A, B, C ] 
###### Fout = [ p1 = 0 ]
###### p0 = ( A  and E0 )   # E0 indica que a porta P0 está com um erro de entrada, preso em 0
###### p1 = ( p0  and B ) 
###### pF2 = ( p1 )


• Estados: São as cláusulas do circuito, as portas com as entradas com falhas ou sem falha.

• Estado inicial: A Cláusula inicial representada pela porta inicial do circuito P0.

• Ações: A formulação mais simples 	define as ações como expandir as portas iniciais P0, P1..Pn respectivamente.

• Modelo de transição: Expandir P0 significa colocar a cláusula P0 em qualquer aparição de P0 no circuito, se faz isso para cada porta respectivamente. Depois resolver P0, P1,...Pn, PSaida1, PSaida2,... pSaidaN, ou seja, fazer a tabela verdade para cada porta respectivamente. Observação primeiro é gerado o circuito sem falha, logo a seguir é expandido o circuito com falha.

• Teste de objetivo: A tabela verdade gerada é igual a tabela verdade do circuito com falha informada pelo usuário.

• Custo de caminho: Cada passo custa 1 e, assim, o custo do caminho é o número de passos do caminho.


## Lendo o circuito de teste
###### - <a href='#8'> voltar ao sumário </a>  

gerarCSV =  1 # 1 Roda o programa com Falha

gerarCSV = 0 # 0 Roda o programa sem Falha


In [874]:
pd.set_option('display.large_repr', 'truncate')
pd.set_option('display.max_columns', 100)

In [875]:

## Execute o programa mais uma  vez dessa vez sem falhas
# Altere testarFalhas para 1 depois para 2 depois 3... depois 10 para executar as 10 falhas para o circuito. Assim que você irá executar o programa 10 vezes.

testarFalhas = 0  # Caso não queira executar as falhas deixe essa linha com 0 e comente a linha abaixo
testarFalhas =2  # caso queira gerar novas falhas de teste mude para 1,2,3,4,5,6,7,8,9 ou 10 para executar a falha correspondente

gerarCSV =  1 # 1 Roda o programa com Falha
# gerarCSV = 0 # 0 Roda o programa sem Falha

# Arquivo sem falhas
semfalhas = "circuito1_semfalha.txt"
comFalha= "circuito.txt"

if gerarCSV==1:
    f = open(comFalha,'r')
#     f = open("pfinalFalha.txt",'r')
    # logicFormCircuit
    texto = f.readlines()
    print(gerarCSV,texto, type(texto), len(texto))
if gerarCSV==0:
    f = open(semfalhas,'r')
    
    f = open("pfinal.txt",'r')
    # logicFormCircuit
    texto = f.readlines()
    print(gerarCSV,texto, type(texto), len(texto))


0 ['var = [ A, B, C, D, E, F]\n', 'p0 = ( not ( ( A or  ( not ( B and  C ) ) or ( B and  C ) )  or  ( ( not ( B and  C ) ) and  ( ( B and  C ) or  ( ( D or E ) ) ) ) ) ) \n', 'p1 = ( not ( ( not ( ( not ( A and  C ) ) and  ( not ( B and  ( not ( C and  D ) ) ) ) ) ) and  ( not ( ( not ( B and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) ) )\n', 'p2 = ( not ( ( not ( ( not ( A and  C ) ) and  ( not ( B and  ( not ( C and  D ) ) ) ) ) ) and  D ) )\n', 'p3 = ( not ( ( ( not ( B and  C ) ) and  ( ( B and  C ) or  ( ( D or E ) ) ) ) and  ( not ( C and  D ) ) ) )\n', 'p4 = ( not ( ( not ( ( not ( B and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) and ( ( ( C and ( A ^  B ) ) or ( B and A ) ) ) ) )\n', 'pF1 = ( ( ( ( A ^  B ) ^ C ) ) )\n', 'pF2 = ( not ( ( not ( C and  D ) ) or  ( not ( ( not ( C and  D ) ) ) ) ) ) \n', 'pF3 = ( not ( ( not ( ( ( not ( B and  C ) ) and  ( ( B and  C ) or  ( ( D or E ) ) ) ) and  ( not ( C and  D ) 

## Testando falhas aleatórias para o circuito.

In [876]:
# f = open(semfalhas,'r')    
# texto = f.readlines()
# texto     

### Pegando variáveis do Circuito

In [877]:
var = []
def getVarCircuit ():
    a =texto[0].replace('=', '').replace('var', '').replace('\n', '').replace("''", '').replace(" ", '').replace("[", '').replace("]", '').replace(",", '')
    ## Lista de variáveis
    var2 = []
    for i in range(len(a)):
#         print(a[i].split(' '))
        var2.append(a[i])
        var.append(a[i])        
    return var2
getVarCircuit()

['A', 'B', 'C', 'D', 'E', 'F']

## Pegando falhas de entrada e saída do circuito

In [878]:
listaPortasFalhasOut = []
def getFalhasOut():
#     auxInOut = texto[1:3]    
    auxInOut = texto[1:2]    
    return auxInOut

getFalhasOut()

['p0 = ( not ( ( A or  ( not ( B and  C ) ) or ( B and  C ) )  or  ( ( not ( B and  C ) ) and  ( ( B and  C ) or  ( ( D or E ) ) ) ) ) ) \n']

### Gerando a lista dos valores de falha para cada porta 

In [879]:
## Mapeando as falhas de Saída das portas
x = getFalhasOut()[0][9:12]
y = getFalhasOut()[0].split(' ')
listFalhaP0 = []
listFalhaP1 = []
listFalhaP2 = []
listFalhaP3 = []
listFalhaP4 = []
listFalhaP5 = []
listFalhaP6 = []
listFalhaP7 = []
listFalhaP8 = []
listFalhaP9 = []
listFalhaP10 = []
listFalhaP11 = []
listFalhaP12 = []
listFalhaP13 = []
listFalhaP14 = []

def getSaidasPortas():
    for i in range(len(y)):
        if y[i] == 'p0' and (y[i+3] == '1' or y[i+3] == '0' ) :  
            listFalhaP0.append(y[i+3])
        if y[i] == 'p0' and (y[i+2] == '1' or y[i+2] == '0' ) :  
            listFalhaP0.append(y[i+2])
        if y[i] == 'p1' and (y[i+3] == '1' or y[i+3] == '0' ) :  
            listFalhaP1.append(y[i+3])
        if y[i] == 'p1' and (y[i+2] == '1' or y[i+2] == '0' ) :  
            listFalhaP1.append(y[i+2])
        if y[i] == 'p2' and (y[i+3] == '1' or y[i+3] == '0' ) :  
            listFalhaP2.append(y[i+3])
        if y[i] == 'p2' and (y[i+2] == '1' or y[i+2] == '0' ) :  
            listFalhaP2.append(y[i+2])
        if y[i] == 'p3' and (y[i+3] == '1' or y[i+3] == '0' ) :  
            listFalhaP3.append(y[i+3])
        if y[i] == 'p3' and (y[i+2] == '1' or y[i+2] == '0' ) :  
            listFalhaP3.append(y[i+2])
        if y[i] == 'p4' and (y[i+3] == '1' or y[i+3] == '0' ) :  
            listFalhaP4.append(y[i+3])
        if y[i] == 'p4' and (y[i+2] == '1' or y[i+2] == '0' ) :  
            listFalhaP4.append(y[i+2])
        if y[i] == 'p5' and (y[i+3] == '1' or y[i+3] == '0' ) :  
            listFalhaP5.append(y[i+3])
        if y[i] == 'p5' and (y[i+2] == '1' or y[i+2] == '0' ) :  
            listFalhaP5.append(y[i+2])
        if y[i] == 'p6' and (y[i+3] == '1' or y[i+3] == '0' ) :  
            listFalhaP6.append(y[i+3])
        if y[i] == 'p6' and (y[i+2] == '1' or y[i+2] == '0' ) :  
            listFalhaP6.append(y[i+2])
        if y[i] == 'p7' and (y[i+3] == '1' or y[i+3] == '0' ) :  
            listFalhaP7.append(y[i+3])
        if y[i] == 'p7' and (y[i+2] == '1' or y[i+2] == '0' ) :  
            listFalhaP7.append(y[i+2])
        if y[i] == 'p8' and (y[i+3] == '1' or y[i+3] == '0' ) :  
            listFalhaP8.append(y[i+3])
        if y[i] == 'p8' and (y[i+2] == '1' or y[i+2] == '0' ) :  
            listFalhaP8.append(y[i+2])
        if y[i] == 'p9' and (y[i+3] == '1' or y[i+3] == '0' ) :  
            listFalhaP9.append(y[i+3])
        if y[i] == 'p9' and (y[i+2] == '1' or y[i+2] == '0' ) :  
            listFalhaP9.append(y[i+2])
        if y[i] == 'p10' and (y[i+3] == '1' or y[i+3] == '0' ) :  
            listFalhaP10.append(y[i+3])
        if y[i] == 'p10' and (y[i+2] == '1' or y[i+2] == '0' ) :              
            listFalhaP10.append(y[i+2])
        if y[i] == 'p11' and (y[i+2] == '1' or y[i+2] == '0' ) :              
            listFalhaP11.append(y[i+2])
        if y[i] == 'p12' and (y[i+2] == '1' or y[i+2] == '0' ) :              
            listFalhaP12.append(y[i+2])
        if y[i] == 'p13' and (y[i+2] == '1' or y[i+2] == '0' ) :              
            listFalhaP13.append(y[i+2])
        if y[i] == 'p14' and (y[i+2] == '1' or y[i+2] == '0' ) :              
            listFalhaP14.append(y[i+2])
            
            
getSaidasPortas()
print('P0', listFalhaP0,'P1',listFalhaP1,'P2',listFalhaP2,'P3',listFalhaP3,'P4',listFalhaP4,'P5',listFalhaP5,'P6',listFalhaP6,'P7',listFalhaP7,'P8',listFalhaP8,'P9',listFalhaP9,'P10',listFalhaP10,'P11',listFalhaP11,'P12',listFalhaP12,'P13',listFalhaP13,'P14',listFalhaP14)


P0 [] P1 [] P2 [] P3 [] P4 [] P5 [] P6 [] P7 [] P8 [] P9 [] P10 [] P11 [] P12 [] P13 [] P14 []


### Pegando o circuito lógico como um mapa para aplicar a um Dataframe

In [880]:
listaPortas = []
listaForm= []
def mapeamentoCircuito():
    lPort = []
    for i in range(2, len(texto)):  # Pegando as portas
        lPort.append(texto[i][0:3])
        listaPortas.append(texto[i][0:3])
    print(lPort)  
    lForm = []
    for i in range(2, len(texto)):
        lForm.append(texto[i][5:])
        listaForm.append(texto[i][5:])
#     print(lForm)
    mapCircuito = dict(zip(lPort, lForm))
#     print("Lista: ",mapCircuito)
    return mapCircuito
mapeamentoCircuito()

['p1 ', 'p2 ', 'p3 ', 'p4 ', 'pF1', 'pF2', 'pF3']


{'p1 ': '( not ( ( not ( ( not ( A and  C ) ) and  ( not ( B and  ( not ( C and  D ) ) ) ) ) ) and  ( not ( ( not ( B and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) ) )\n',
 'p2 ': '( not ( ( not ( ( not ( A and  C ) ) and  ( not ( B and  ( not ( C and  D ) ) ) ) ) ) and  D ) )\n',
 'p3 ': '( not ( ( ( not ( B and  C ) ) and  ( ( B and  C ) or  ( ( D or E ) ) ) ) and  ( not ( C and  D ) ) ) )\n',
 'p4 ': '( not ( ( not ( ( not ( B and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) and ( ( ( C and ( A ^  B ) ) or ( B and A ) ) ) ) )\n',
 'pF1': ' ( ( ( ( A ^  B ) ^ C ) ) )\n',
 'pF2': ' ( not ( ( not ( C and  D ) ) or  ( not ( ( not ( C and  D ) ) ) ) ) ) \n',
 'pF3': ' ( not ( ( not ( ( ( not ( B and  C ) ) and  ( ( B and  C ) or  ( ( D or E ) ) ) ) and  ( not ( C and  D ) ) ) ) and ( not ( ( not ( ( not ( B and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) and ( ( ( C and ( A ^  B ) ) or ( B an

# Pegando o circuito e colocando em um csv

In [881]:
# Pegando o circuito e colocando em um CSV
dfCircuito = pd.DataFrame(data=mapeamentoCircuito(), index=[0])
dfCircuito

['p1 ', 'p2 ', 'p3 ', 'p4 ', 'pF1', 'pF2', 'pF3']


,p1,p2,p3,p4,pF1,pF2,pF3
0,( not ( ( not ( ( not ( A and C ) ) and ( no...,( not ( ( not ( ( not ( A and C ) ) and ( no...,( not ( ( ( not ( B and C ) ) and ( ( B and ...,( not ( ( not ( ( not ( B and ( not ( C and ...,( ( ( ( A ^ B ) ^ C ) ) )\n,( not ( ( not ( C and D ) ) or ( not ( ( no...,( not ( ( not ( ( ( not ( B and C ) ) and (...


# Colocando o circuito como uma lista para repassar para a função que irá gerar a tabela verdade

In [882]:
# Colocando o circuito como uma lista para repassar para a função que irá gerar a tabela verdade
listCircuito = dfCircuito.values.tolist()
listCircuito = list(chain.from_iterable(listCircuito)) # gerando uma lista de uma lista de lista
listCircuito

['( not ( ( not ( ( not ( A and  C ) ) and  ( not ( B and  ( not ( C and  D ) ) ) ) ) ) and  ( not ( ( not ( B and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) ) )\n',
 '( not ( ( not ( ( not ( A and  C ) ) and  ( not ( B and  ( not ( C and  D ) ) ) ) ) ) and  D ) )\n',
 '( not ( ( ( not ( B and  C ) ) and  ( ( B and  C ) or  ( ( D or E ) ) ) ) and  ( not ( C and  D ) ) ) )\n',
 '( not ( ( not ( ( not ( B and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) and ( ( ( C and ( A ^  B ) ) or ( B and A ) ) ) ) )\n',
 ' ( ( ( ( A ^  B ) ^ C ) ) )\n',
 ' ( not ( ( not ( C and  D ) ) or  ( not ( ( not ( C and  D ) ) ) ) ) ) \n',
 ' ( not ( ( not ( ( ( not ( B and  C ) ) and  ( ( B and  C ) or  ( ( D or E ) ) ) ) and  ( not ( C and  D ) ) ) ) and ( not ( ( not ( ( not ( B and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) and ( ( ( C and ( A ^  B ) ) or ( B and A ) ) ) ) ) ) )']

#############################################################################################################
## Colocando as falhas no cirtuito: Gerando falhas 
#############################################################################################################
###### - <a href='#8'> voltar ao sumário </a>  


In [883]:
def gerandoFalhasParaOCircuito1():
#     print('listCircuito ',listCircuito)
    for feature in dfCircuito.columns:
        for i in range(len(listCircuito)):
            dfCircuito[feature].replace('A', 'E0', regex=True, inplace=True)
            print('Falha1  ',dfCircuito[feature])

def gerandoFalhasParaOCircuito2():
    for feature in dfCircuito.columns:
        for i in range(len(listCircuito)):
            dfCircuito[feature].replace('B', 'E0', regex=True, inplace=True)
            print('Falha2  ',dfCircuito[feature])

def gerandoFalhasParaOCircuito3():
    for feature in dfCircuito.columns:
        for i in range(len(listCircuito)):
            dfCircuito[feature].replace('C', 'E0', regex=True, inplace=True)
            print('Falha3  ',dfCircuito[feature])

def gerandoFalhasParaOCircuito4():
    for feature in dfCircuito.columns:
        for i in range(len(listCircuito)):
            dfCircuito[feature].replace('A', 'E1', regex=True, inplace=True)
            print('Falha4  ',dfCircuito[feature])

def gerandoFalhasParaOCircuito5():
    for feature in dfCircuito.columns:
        for i in range(len(listCircuito)):
            dfCircuito[feature].replace('B', 'E1', regex=True, inplace=True)
            print('Falha5  ',dfCircuito[feature])

def gerandoFalhasParaOCircuito6():
    for feature in dfCircuito.columns:
        for i in range(len(listCircuito)):
            dfCircuito[feature].replace('C', 'E1', regex=True, inplace=True)
            print('Falha6  ',dfCircuito[feature])

def gerandoFalhasParaOCircuito7():
    for feature in dfCircuito.columns:
        for i in range(len(listCircuito)):
            dfCircuito[feature].replace('A', 'E1', regex=True, inplace=True)
            dfCircuito[feature].replace('B', 'E1', regex=True, inplace=True)
            print('Falha7  ',dfCircuito[feature])

def gerandoFalhasParaOCircuito8():
    for feature in dfCircuito.columns:
        for i in range(len(listCircuito)):
            dfCircuito[feature].replace('A', 'E0', regex=True, inplace=True)
            dfCircuito[feature].replace('B', 'E0', regex=True, inplace=True)
            print('Falha8  ',dfCircuito[feature])

def gerandoFalhasParaOCircuito9():
    for feature in dfCircuito.columns:
        for i in range(len(listCircuito)):
            dfCircuito[feature].replace('A', 'E0', regex=True, inplace=True)
            dfCircuito[feature].replace('B', 'E1', regex=True, inplace=True)
            print('Falha9  ',dfCircuito[feature])

def gerandoFalhasParaOCircuito10():
    for feature in dfCircuito.columns:
        for i in range(len(listCircuito)):
            dfCircuito[feature].replace('A', 'E1', regex=True, inplace=True)
            dfCircuito[feature].replace('B', 'E1', regex=True, inplace=True)
            print('Falha10  ',dfCircuito[feature])

if testarFalhas == 1:     
    gerandoFalhasParaOCircuito1()
if testarFalhas == 2:     
    gerandoFalhasParaOCircuito2()
if testarFalhas == 3:     
    gerandoFalhasParaOCircuito3()
if testarFalhas == 4:     
    gerandoFalhasParaOCircuito4()
if testarFalhas == 5:     
    gerandoFalhasParaOCircuito5()
if testarFalhas == 6:     
    gerandoFalhasParaOCircuito6()
if testarFalhas == 7:     
    gerandoFalhasParaOCircuito7()
if testarFalhas == 8:     
    gerandoFalhasParaOCircuito8()
if testarFalhas == 9:     
    gerandoFalhasParaOCircuito9()
if testarFalhas == 10:     
    gerandoFalhasParaOCircuito10()


Falha2   0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
Falha2   0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
Falha2   0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
Falha2   0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
Falha2   0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
Falha2   0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
Falha2   0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
Falha2   0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
Falha2   0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
Falha2   0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
Falha2   0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
Falha2   0    ( not (

In [884]:
# Colocando o circuito como uma lista para repassar para a função que irá gerar a tabela verdade
listCircuito = dfCircuito.values.tolist()
listCircuito = list(chain.from_iterable(listCircuito)) # gerando uma lista de uma lista de lista
listCircuito

['( not ( ( not ( ( not ( A and  C ) ) and  ( not ( E0 and  ( not ( C and  D ) ) ) ) ) ) and  ( not ( ( not ( E0 and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) ) )\n',
 '( not ( ( not ( ( not ( A and  C ) ) and  ( not ( E0 and  ( not ( C and  D ) ) ) ) ) ) and  D ) )\n',
 '( not ( ( ( not ( E0 and  C ) ) and  ( ( E0 and  C ) or  ( ( D or E ) ) ) ) and  ( not ( C and  D ) ) ) )\n',
 '( not ( ( not ( ( not ( E0 and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) and ( ( ( C and ( A ^  E0 ) ) or ( E0 and A ) ) ) ) )\n',
 ' ( ( ( ( A ^  E0 ) ^ C ) ) )\n',
 ' ( not ( ( not ( C and  D ) ) or  ( not ( ( not ( C and  D ) ) ) ) ) ) \n',
 ' ( not ( ( not ( ( ( not ( E0 and  C ) ) and  ( ( E0 and  C ) or  ( ( D or E ) ) ) ) and  ( not ( C and  D ) ) ) ) and ( not ( ( not ( ( not ( E0 and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) and ( ( ( C and ( A ^  E0 ) ) or ( E0 and A ) ) ) ) ) ) )']

# - <a id='#2'>2. Aplicando cada fórmula nas portas</a>  
###### - <a href='#8'> voltar ao sumário </a>  

Essa é uma etapa importante do problema, pois aplicamos as substituições necessárias das falhas, ou seja, propagamos as falhas pelas portas e aplicamos as falhas no circuito se o mesmo tiver falhas.



In [885]:
def aplicaClausula():
    # Substituindo e gerando o circuito. Circuito de Teste
    for feature in dfCircuito.columns:
        for i in range(len(listCircuito)):
            print(dfCircuito[feature])
            dfCircuito[feature].replace(dfCircuito.columns[i], listCircuito[i], regex=True, inplace=True)
            dfCircuito[feature].replace("\n", "", regex=True, inplace=True)            
            dfCircuito[feature].replace(dfCircuito.columns[i], listCircuito[i], regex=True, inplace=True)
            dfCircuito[feature].replace("\n", "", regex=True, inplace=True)            
# aplicaClausula()

In [886]:
listCircuito[0] == dfCircuito.values[0][0]  # Circuito Lógico

def aplicaClausulaComFalhas():
    # Substituindo e gerando o circuito. Circuito de Teste
    for feature in dfCircuito.columns:
        for i in range(len(listCircuito)):
            
            ## Pegando os erros de entrada para cada porta do circuito
            if 'E0' in listCircuito[i]:
                listCircuito[i].replace('E0', '0' )
                dfCircuito[feature].replace('E0', '0', regex=True, inplace=True)

#                 print('E0')
            if 'E1' in listCircuito[i]:
                listCircuito[i].replace('E1', '1' )                
                dfCircuito[feature].replace('E1', '1', regex=True, inplace=True)
#                 print('E1')
            
            if len(listFalhaP0)>0: 
                dfCircuito[feature].replace(listCircuito[0], listFalhaP0[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p0', listFalhaP0[0], regex=True, inplace=True)
#                 print('sucesso p0')
            if len(listFalhaP1)>0:  # Se temos falhas de Saida em P1 --len(listFalhaP0)
                dfCircuito[feature].replace(listCircuito[1], listFalhaP1[0], regex=True, inplace=True)  # listCircuito[1], representa a expressão do circuito para a porta 1
                dfCircuito[feature].replace('p1', listFalhaP1[0], regex=True, inplace=True)

            if len(listFalhaP2)>0:  
                dfCircuito[feature].replace(listCircuito[2], listFalhaP2[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p2', listFalhaP2[0], regex=True, inplace=True)
                
            if len(listFalhaP3)>0:  
                dfCircuito[feature].replace(listCircuito[3], listFalhaP3[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p3', listFalhaP3[0], regex=True, inplace=True)
                
            if len(listFalhaP4)>0:  
                dfCircuito[feature].replace(listCircuito[4], listFalhaP4[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p4', listFalhaP4[0], regex=True, inplace=True)
            if len(listFalhaP5)>0:  
                dfCircuito[feature].replace(listCircuito[5], listFalhaP5[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p5', listFalhaP5[0], regex=True, inplace=True)
                
            if len(listFalhaP6)>0:  
                dfCircuito[feature].replace(listCircuito[6], listFalhaP6[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p6', listFalhaP6[0], regex=True, inplace=True)
                
            if len(listFalhaP7)>0:  
                dfCircuito[feature].replace(listCircuito[7], listFalhaP7[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p7', listFalhaP7[0], regex=True, inplace=True)
                
            if len(listFalhaP8)>0:  
                dfCircuito[feature].replace(listCircuito[8], listFalhaP8[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p8', listFalhaP8[0], regex=True, inplace=True)

            if len(listFalhaP9)>0:  
                dfCircuito[feature].replace(listCircuito[9], listFalhaP9[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p9', listFalhaP9[0], regex=True, inplace=True)

            if len(listFalhaP10)>0:  
                dfCircuito[feature].replace(listCircuito[8], listFalhaP10[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p10', listFalhaP10[10], regex=True, inplace=True)

            if len(listFalhaP11)>0:  
                dfCircuito[feature].replace(listCircuito[11], listFalhaP11[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p11', listFalhaP10[0], regex=True, inplace=True)
            if len(listFalhaP12)>0:  
                dfCircuito[feature].replace(listCircuito[12], listFalhaP11[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p12', listFalhaP12[0], regex=True, inplace=True)
            if len(listFalhaP13)>0:  
                dfCircuito[feature].replace(listCircuito[13], listFalhaP13[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p13', listFalhaP13[0], regex=True, inplace=True)
            if len(listFalhaP14)>0:  
                dfCircuito[feature].replace(listCircuito[14], listFalhaP14[0], regex=True, inplace=True)
                dfCircuito[feature].replace('p14', listFalhaP14[0], regex=True, inplace=True)

if testarFalhas>0:
    aplicaClausulaComFalhas()
# aplicaClausulaComFalhas()

In [887]:

circuitoOriginalVar = []
circuitoOriginal = []
# Gerando o circuito 1 original
circuitoOriginalVar.append('var = ')          
for i in range(len(var)):
    circuitoOriginalVar.append(var[i])          
for feature in dfCircuito.columns:
    for i in range(len(listCircuito)):
        circuitoOriginal.append(dfCircuito.columns[i]+' = '+ listCircuito[i])  
        
        
circuitoOriginalVar = str(circuitoOriginalVar).replace("'", '').replace("  ", ' ').replace("[var", 'var ').replace(' , ',' [ ').replace('")', '').replace("('", '')
c = str(circuitoOriginal).replace("['", '').replace("\n", '').replace("']", '') 
# print(c)

if gerarCSV ==0:
    aplicaClausula()    
if gerarCSV ==1:
    aplicaClausulaComFalhas() 
    

0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) 

## Salvando o csv com o circuito

In [888]:

if gerarCSV ==1:
#     AplicarFalhaCircuito()  # depois
    aplicaClausulaComFalhas()
#     dfCircuito.to_csv('circuito1ComFalha.csv', index=False)   # Caso queira verificar o circuito no formato inicial como uma tabela verdade
else:   
    aplicaClausula()
#     dfCircuito.to_csv('circuito1SemFalha.csv', index=False)

0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p1 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) ) and  ( no...
Name: p2 , dtype: object
0    ( not ( ( not ( ( not ( A and  C ) 

## Função para gerar a tabela verdade.  
aqui você pode escolher qual tabela verdade gerar 1. 

Essa é uma forma simplificada de gerar as tabelas do circuito com falha e sem falha. O controle é feito pela variável 'gerarCSV'

em que o usuário escolhe assim que executa o programa.

In [889]:
### Informe o nome do arquivo para saída do Circuito
if gerarCSV == 1:
    saidaCircuitoTxt = 'saidaCircuitoTxt1Falha'
    saidaCircuitoCsv = 'saidaCircuitoCSV1Falha'
else:
    saidaCircuitoTxt = 'saidaCircuitoTxt1SemFalha'
    saidaCircuitoCsv = 'saidaCircuitoCSV1SemFalha'
    

# - <a href='#3'>3. As classes  TruthsGenerates, são responsáveis por gerar todas as combinações binárias da tabela verdade.</a>  
###### - <a href='#8'> voltar ao sumário </a>  



## E gerar o arquivo de texto, csv da tabela verdade.

In [890]:
#### quando geraFalhasRand=1 o programa gera as falhas
class Gob(object):
    pass

class TruthsGenerateF3(object): # porta 3
    def __init__(self, base=None, phrases=None, ints=True):
        if not base:
            raise Exception('Base items are required')
        self.base = base
        self.phrases = phrases or []
        self.ints = ints

        # generate the sets of booleans for the bases
        self.base_conditions = list(itertools.product([False, True],
                                                      repeat=len(base)))

        # regex to match whole words defined in self.bases
        # used to add object context to variables in self.phrases
        self.p = re.compile(r'(?<!\w)(' + '|'.join(self.base) + ')(?!\w)')

    def calculate(self, *args):
        # store bases in an object context
        g = Gob()
        for a, b in zip(self.base, args):
            setattr(g, a, b)

        # add object context to any base variables in self.phrases
        # then evaluate each
        eval_phrases = []
        for item in self.phrases:
            item = self.p.sub(r'g.\1', item)
            eval_phrases.append(eval(item))

        # add the bases and evaluated phrases to create a single row
        row = [getattr(g, b) for b in self.base] + eval_phrases
        if self.ints:
            return [int(item) for item in row]
        else:
            return row

    def __str__(self):
        t = PrettyTable(self.base + self.phrases)
        for conditions_set in self.base_conditions:
            t.add_row(self.calculate(*conditions_set))
                         # Criando e escrevendo em arquivos de texto (modo 'w').
            arquivo2 = open('SaidasFalhasS3.txt','a')
            arquivo2.write(str(t))
            arquivo2.close()
            df2 = pd.read_html(t.get_html_string())[0]
            df2.columns = df2.iloc[0]
            df2 = df2.reindex(df2.index.drop(0))
            df2.to_csv("SaidasFalhasS3.csv" , index=False)


        return str(t)# t.get_html_string()  #str(t)

class TruthsGenerateF1_tres_tres(object):
    def __init__(self, base=None, phrases=None, ints=True):
        if not base:
            raise Exception('Base items are required')
        self.base = base
        self.phrases = phrases or []
        self.ints = ints

        # generate the sets of booleans for the bases
        self.base_conditions = list(itertools.product([False, True],
                                                      repeat=len(base)))

        # regex to match whole words defined in self.bases
        # used to add object context to variables in self.phrases
        self.p = re.compile(r'(?<!\w)(' + '|'.join(self.base) + ')(?!\w)')

    def calculate(self, *args):
        # store bases in an object context
        g = Gob()
        for a, b in zip(self.base, args):
            setattr(g, a, b)

        # add object context to any base variables in self.phrases
        # then evaluate each
        eval_phrases = []
        for item in self.phrases:
            item = self.p.sub(r'g.\1', item)
            eval_phrases.append(eval(item))

        # add the bases and evaluated phrases to create a single row
        row = [getattr(g, b) for b in self.base] + eval_phrases
        if self.ints:
            return [int(item) for item in row]
        else:
            return row

    def __str__(self):
        t = PrettyTable(self.base + self.phrases)
        for conditions_set in self.base_conditions:
            t.add_row(self.calculate(*conditions_set))
                         # Criando e escrevendo em arquivos de texto (modo 'w').
            arquivo2 = open('SaidasFalhasS1_3x3.txt','a')
            arquivo2.write(str(t))
            arquivo2.close()
            df2 = pd.read_html(t.get_html_string())[0]
            df2.columns = df2.iloc[0]
            df2 = df2.reindex(df2.index.drop(0))
            df2.to_csv("SaidasFalhasS1_3x3.csv" , index=False)


        return str(t)# t.get_html_string()  #str(t)


class TruthsGenerateF2_tres_tres(object):
    def __init__(self, base=None, phrases=None, ints=True):
        if not base:
            raise Exception('Base items are required')
        self.base = base
        self.phrases = phrases or []
        self.ints = ints

        # generate the sets of booleans for the bases
        self.base_conditions = list(itertools.product([False, True],
                                                      repeat=len(base)))

        # regex to match whole words defined in self.bases
        # used to add object context to variables in self.phrases
        self.p = re.compile(r'(?<!\w)(' + '|'.join(self.base) + ')(?!\w)')

    def calculate(self, *args):
        # store bases in an object context
        g = Gob()
        for a, b in zip(self.base, args):
            setattr(g, a, b)

        # add object context to any base variables in self.phrases
        # then evaluate each
        eval_phrases = []
        for item in self.phrases:
            item = self.p.sub(r'g.\1', item)
            eval_phrases.append(eval(item))

        # add the bases and evaluated phrases to create a single row
        row = [getattr(g, b) for b in self.base] + eval_phrases
        if self.ints:
            return [int(item) for item in row]
        else:
            return row

    def __str__(self):
        t = PrettyTable(self.base + self.phrases)
        for conditions_set in self.base_conditions:
            t.add_row(self.calculate(*conditions_set))
                         # Criando e escrevendo em arquivos de texto (modo 'w').
            arquivo2 = open('SaidasFalhasS2_3x3.txt','a')
            arquivo2.write(str(t))
            arquivo2.close()
            df2 = pd.read_html(t.get_html_string())[0]
            df2.columns = df2.iloc[0]
            df2 = df2.reindex(df2.index.drop(0))
            df2.to_csv("SaidasFalhasS2_3x3.csv" , index=False)


        return str(t)# t.get_html_string()  #str(t)


class TruthsGenerateF1_dois_dois(object):
    def __init__(self, base=None, phrases=None, ints=True):
        if not base:
            raise Exception('Base items are required')
        self.base = base
        self.phrases = phrases or []
        self.ints = ints

        # generate the sets of booleans for the bases
        self.base_conditions = list(itertools.product([False, True],
                                                      repeat=len(base)))

        # regex to match whole words defined in self.bases
        # used to add object context to variables in self.phrases
        self.p = re.compile(r'(?<!\w)(' + '|'.join(self.base) + ')(?!\w)')

    def calculate(self, *args):
        # store bases in an object context
        g = Gob()
        for a, b in zip(self.base, args):
            setattr(g, a, b)

        # add object context to any base variables in self.phrases
        # then evaluate each
        eval_phrases = []
        for item in self.phrases:
            item = self.p.sub(r'g.\1', item)
            eval_phrases.append(eval(item))

        # add the bases and evaluated phrases to create a single row
        row = [getattr(g, b) for b in self.base] + eval_phrases
        if self.ints:
            return [int(item) for item in row]
        else:
            return row

    def __str__(self):
        t = PrettyTable(self.base + self.phrases)
        for conditions_set in self.base_conditions:
            t.add_row(self.calculate(*conditions_set))
                         # Criando e escrevendo em arquivos de texto (modo 'w').
            arquivo2 = open('SaidasFalhasS1_2x2.txt','a')
            arquivo2.write(str(t))
            arquivo2.close()
            df2 = pd.read_html(t.get_html_string())[0]
            df2.columns = df2.iloc[0]
            df2 = df2.reindex(df2.index.drop(0))
            df2.to_csv("SaidasFalhasS1_2x2.csv" , index=False)


        return str(t)# t.get_html_string()  #str(t)

class TruthsGenerateF2_dois_dois(object):
    def __init__(self, base=None, phrases=None, ints=True):
        if not base:
            raise Exception('Base items are required')
        self.base = base
        self.phrases = phrases or []
        self.ints = ints

        # generate the sets of booleans for the bases
        self.base_conditions = list(itertools.product([False, True],
                                                      repeat=len(base)))

        # regex to match whole words defined in self.bases
        # used to add object context to variables in self.phrases
        self.p = re.compile(r'(?<!\w)(' + '|'.join(self.base) + ')(?!\w)')

    def calculate(self, *args):
        # store bases in an object context
        g = Gob()
        for a, b in zip(self.base, args):
            setattr(g, a, b)

        # add object context to any base variables in self.phrases
        # then evaluate each
        eval_phrases = []
        for item in self.phrases:
            item = self.p.sub(r'g.\1', item)
            eval_phrases.append(eval(item))

        # add the bases and evaluated phrases to create a single row
        row = [getattr(g, b) for b in self.base] + eval_phrases
        if self.ints:
            return [int(item) for item in row]
        else:
            return row

    def __str__(self):
        t = PrettyTable(self.base + self.phrases)
        for conditions_set in self.base_conditions:
            t.add_row(self.calculate(*conditions_set))
                         # Criando e escrevendo em arquivos de texto (modo 'w').
            arquivo2 = open('SaidasFalhasS2_2x2.txt','a')
            arquivo2.write(str(t))
            arquivo2.close()
            df2 = pd.read_html(t.get_html_string())[0]
            df2.columns = df2.iloc[0]
            df2 = df2.reindex(df2.index.drop(0))
            df2.to_csv("SaidasFalhasS2_2x2.csv" , index=False)


        return str(t)# t.get_html_string()  #str(t)
    

class TruthsGenerateF2(object):
    def __init__(self, base=None, phrases=None, ints=True):
        if not base:
            raise Exception('Base items are required')
        self.base = base
        self.phrases = phrases or []
        self.ints = ints

        # generate the sets of booleans for the bases
        self.base_conditions = list(itertools.product([False, True],
                                                      repeat=len(base)))

        # regex to match whole words defined in self.bases
        # used to add object context to variables in self.phrases
        self.p = re.compile(r'(?<!\w)(' + '|'.join(self.base) + ')(?!\w)')

    def calculate(self, *args):
        # store bases in an object context
        g = Gob()
        for a, b in zip(self.base, args):
            setattr(g, a, b)

        # add object context to any base variables in self.phrases
        # then evaluate each
        eval_phrases = []
        for item in self.phrases:
            item = self.p.sub(r'g.\1', item)
            eval_phrases.append(eval(item))

        # add the bases and evaluated phrases to create a single row
        row = [getattr(g, b) for b in self.base] + eval_phrases
        if self.ints:
            return [int(item) for item in row]
        else:
            return row

    def __str__(self):
        t = PrettyTable(self.base + self.phrases)
        for conditions_set in self.base_conditions:
            t.add_row(self.calculate(*conditions_set))
                         # Criando e escrevendo em arquivos de texto (modo 'w').
            arquivo2 = open('SaidasFalhasS2.txt','a')
            arquivo2.write(str(t))
            arquivo2.close()
            df2 = pd.read_html(t.get_html_string())[0]
            df2.columns = df2.iloc[0]
            df2 = df2.reindex(df2.index.drop(0))
            df2.to_csv("SaidasFalhasS2.csv" , index=False)


        return str(t)# t.get_html_string()  #str(t)
    
    
class TruthsGenerateF1(object):
    def __init__(self, base=None, phrases=None, ints=True):
        if not base:
            raise Exception('Base items are required')
        self.base = base
        self.phrases = phrases or []
        self.ints = ints

        # generate the sets of booleans for the bases
        self.base_conditions = list(itertools.product([False, True],
                                                      repeat=len(base)))

        # regex to match whole words defined in self.bases
        # used to add object context to variables in self.phrases
        self.p = re.compile(r'(?<!\w)(' + '|'.join(self.base) + ')(?!\w)')

    def calculate(self, *args):
        # store bases in an object context
        g = Gob()
        for a, b in zip(self.base, args):
            setattr(g, a, b)

        # add object context to any base variables in self.phrases
        # then evaluate each
        eval_phrases = []
        for item in self.phrases:
            item = self.p.sub(r'g.\1', item)
            eval_phrases.append(eval(item))

        # add the bases and evaluated phrases to create a single row
        row = [getattr(g, b) for b in self.base] + eval_phrases
        if self.ints:
            return [int(item) for item in row]
        else:
            return row

    def __str__(self):
        t = PrettyTable(self.base + self.phrases)
        for conditions_set in self.base_conditions:
            t.add_row(self.calculate(*conditions_set))
                # Criando e escrevendo em arquivos de texto (modo 'w').
            arquivo3 = open('SaidasFalhasS1.txt','a')
            arquivo3.write(str(t))
            arquivo3.close()
            df3 = pd.read_html(t.get_html_string())[0]
            df3.columns = df3.iloc[0]
            df3 = df3.reindex(df3.index.drop(0))
            df3.to_csv("SaidasFalhasS1.csv" , index=False)

        return str(t)# t.get_html_string()  #str(t)

class Truths(object):
    def __init__(self, base=None, phrases=None, ints=True):
        if not base:
            raise Exception('Base items are required')
        self.base = base
        self.phrases = phrases or []
        self.ints = ints

        # generate the sets of booleans for the bases
        self.base_conditions = list(itertools.product([False, True],
                                                      repeat=len(base)))

        # regex to match whole words defined in self.bases
        # used to add object context to variables in self.phrases
        self.p = re.compile(r'(?<!\w)(' + '|'.join(self.base) + ')(?!\w)')

    def calculate(self, *args):
        # store bases in an object context
        g = Gob()
        for a, b in zip(self.base, args):
            setattr(g, a, b)

        # add object context to any base variables in self.phrases
        # then evaluate each
        eval_phrases = []
        for item in self.phrases:
            item = self.p.sub(r'g.\1', item)
            eval_phrases.append(eval(item))

        # add the bases and evaluated phrases to create a single row
        row = [getattr(g, b) for b in self.base] + eval_phrases
        if self.ints:
            return [int(item) for item in row]
        else:
            return row

    def __str__(self):
        t = PrettyTable(self.base + self.phrases)
        for conditions_set in self.base_conditions:
            t.add_row(self.calculate(*conditions_set))
            # Criando e escrevendo em arquivos de texto (modo 'w'). # Falhas
            arquivo = open(saidaCircuitoTxt+'.txt','w')
            arquivo.write(str(t))
            arquivo.close()
            df = pd.read_html(t.get_html_string())[0]
            df.columns = df.iloc[0]
            df = df.reindex(df.index.drop(0))
            df.to_csv(saidaCircuitoCsv+".csv" , index=False)
        return str(t)# t.get_html_string()  #str(t)


# Colocando o circuito como uma lista para repassar para a função que irá gerar a tabela verdade

In [891]:
# Colocando o circuito como uma lista para repassar para a função que irá gerar a tabela verdade
listCircuito = dfCircuito.values.tolist()
listCircuito = list(chain.from_iterable(listCircuito))
# listCircuito

#### Variáveis do circuito, regras de cada porta.

#### Substituindo a falha no circuito. 

### A função AplicarFalhaCircuito() é responsável por aplicar as falhas de saída do circuio caso o circuito tenha falhas de saída.

In [892]:
def AplicarFalhaCircuito():
    # O passo trivial é que se existir mais portas no listCircuito ele será mapeado e o len(listCircuito) não excederá o index
    for i in range(len(listCircuito)):
        if len(listFalhaP0)>0:
            listCircuito[0] = listFalhaP0[0]
        if len(listFalhaP1)>0:
            listCircuito[1] = listFalhaP1[0]
        if len(listFalhaP2)>0:
            listCircuito[2] = listFalhaP2[0]
        if len(listFalhaP3)>0:
            listCircuito[3] = listFalhaP3[0]
        if len(listFalhaP4)>0:
            listCircuito[4] = listFalhaP4[0]
        if len(listFalhaP5)>0:
            listCircuito[5] = listFalhaP5[0]
        if len(listFalhaP6)>0:
            listCircuito[6] = listFalhaP6[0]
        if len(listFalhaP7)>0:
            listCircuito[7] = listFalhaP7[0]
        if len(listFalhaP8)>0:
            listCircuito[8] = listFalhaP8[0]
        if len(listFalhaP9)>0:
            listCircuito[9] = listFalhaP9[0]
        if len(listFalhaP10)>0:
            listCircuito[10] = listFalhaP10[0]
if gerarCSV ==0:  
    #### Funções que geram as falhas do circuito.
    aplicaClausulaComFalhas()
    AplicarFalhaCircuito()
#     data2 = pd.read_csv('circuito1ComFalha.csv')
#     data2.head(10)  # vendo os primeiros valores da tabela verdade
else:  
    #### Funções que geram as falhas do circuito.
    aplicaClausula()
#     data2 = pd.read_csv('circuito1SemFalha.csv')
#     data2.head(10)  # vendo os primeiros valores da tabela verdade

## A próxima etapa após capturar as falhas do circuito é gerar combinações de falhas aleatórias. 

### Para isso as funções abaixo, simularFalhasPorta13x3, simularFalhasPorta12x2, simularFalhasPorta11x1... simulam falhas aleatórias combinando 1 a 1, 2 a 2 e 3 a 3.

A estratégia para gerar combinações das falhas é capturar os índices das variáveis do circuito e depois verificar a ocorrências desses íncides no circuito e aplicar 0 ou 1 nos índices das variáveis.

In [893]:

listaIndicesVar = []
listaIndicesVarDoisDois = []

listaSaidasCircuitoF1 = []
saidasCircuito = listCircuito[::-1] # Pegando as saídas Finais dos Circuitos
saidasCircuitoF1 = saidasCircuito[1]

for i in range(len(saidasCircuitoF1)): # pegando os índices das variáveis para simular falhas de entrada no circuito
    for j in range(len(var)):
        if saidasCircuitoF1[i] == var[j]:
            listaIndicesVar.append(i)

for subset in permutations(listaIndicesVar, 3):
    listaIndicesVarDoisDois.append(subset)
    
# for i in range(len(listaIndicesVarDoisDois)):
#     print(listaIndicesVarDoisDois[i], len(listaIndicesVarDoisDois[i]))
#     print(print(listaIndicesVarDoisDois[i])


In [894]:
def simularFalhasPorta13x3():
        # Simular 10 falhas aleatórias no circuito
    listaIndicesVar = []
    listaSaidasCircuitoF2 = []
    listaIndicesVarTreEmTres = []

    saidasCircuito = listCircuito[::-1] # Pegando as saídas Finais dos Circuitos
    saidasCircuitoF2 = saidasCircuito[1] # mudando a porta do circuito
#     print(saidasCircuitoF1)
    for i in range(len(saidasCircuitoF2)): # pegando os índices das variáveis para simular falhas de entrada no circuito
        for j in range(len(var)):
            if saidasCircuitoF2[i] == var[j]:
                listaIndicesVar.append(i)

    # Pegando todas as combinações 2 a 2 e gerando 10 falhas aleatórias no circuito
    for subset in permutations(listaIndicesVar, 2):
        listaIndicesVarTreEmTres.append(subset)
        
    for i in range(len(listaIndicesVarTreEmTres)):        
        for j in range(len(listaIndicesVarTreEmTres[i])):                
            repP0 = saidasCircuitoF2.replace(saidasCircuitoF2[listaIndicesVarTreEmTres[j][0]], '0' )
            listaSaidasCircuitoF2.append(repP0)
            repP0_ = saidasCircuitoF2.replace(saidasCircuitoF2[listaIndicesVarTreEmTres[j][1]], '1' )
            listaSaidasCircuitoF2.append(repP0_)  
            repP0_2 = saidasCircuitoF2.replace(saidasCircuitoF2[listaIndicesVarTreEmTres[i][0]], '1' )
            listaSaidasCircuitoF2.append(repP0_2)  
        
    return sorted(set(listaSaidasCircuitoF2))

def simularFalhasPorta23x3():
        # Simular 10 falhas aleatórias no circuito
    listaIndicesVar = []
    listaSaidasCircuitoF2 = []
    listaIndicesVarTreEmTres = []

    saidasCircuito = listCircuito[::-1] # Pegando as saídas Finais dos Circuitos
    saidasCircuitoF2 = saidasCircuito[0]
#     print(saidasCircuitoF1)
    for i in range(len(saidasCircuitoF2)): # pegando os índices das variáveis para simular falhas de entrada no circuito
        for j in range(len(var)):
            if saidasCircuitoF2[i] == var[j]:
                listaIndicesVar.append(i)

    # Pegando todas as combinações 2 a 2 e gerando 10 falhas aleatórias no circuito
    for subset in permutations(listaIndicesVar, 2):
        listaIndicesVarTreEmTres.append(subset)
        
    for i in range(len(listaIndicesVarTreEmTres)):        
        for j in range(len(listaIndicesVarTreEmTres[i])):                
            repP0 = saidasCircuitoF2.replace(saidasCircuitoF2[listaIndicesVarTreEmTres[j][0]], '0' )
            listaSaidasCircuitoF2.append(repP0)
            repP0_ = saidasCircuitoF2.replace(saidasCircuitoF2[listaIndicesVarTreEmTres[j][1]], '1' )
            listaSaidasCircuitoF2.append(repP0_)  
            repP0_2 = saidasCircuitoF2.replace(saidasCircuitoF2[listaIndicesVarTreEmTres[i][0]], '1' )
            listaSaidasCircuitoF2.append(repP0_2)  
        
    return sorted(set(listaSaidasCircuitoF2))

def simularFalhasPorta22x2():
        # Simular 10 falhas aleatórias no circuito
    listaIndicesVar = []
    listaSaidasCircuitoF2 = []
    listaIndicesVarDoisDois = []

    saidasCircuito = listCircuito[::-1] # Pegando as saídas Finais dos Circuitos
    saidasCircuitoF2 = saidasCircuito[0]
#     print(saidasCircuitoF1)
    for i in range(len(saidasCircuitoF2)): # pegando os índices das variáveis para simular falhas de entrada no circuito
        for j in range(len(var)):
            if saidasCircuitoF2[i] == var[j]:
                listaIndicesVar.append(i)

    # Pegando todas as combinações 2 a 2 e gerando 10 falhas aleatórias no circuito
    for subset in permutations(listaIndicesVar, 2):
        listaIndicesVarDoisDois.append(subset)
        
    for i in range(len(listaIndicesVarDoisDois)):        
        repP0 = saidasCircuitoF2.replace(saidasCircuitoF2[listaIndicesVarDoisDois[i][0]], '0' )
        listaSaidasCircuitoF2.append(repP0)
#         repP0_2 = saidasCircuitoF2.replace(saidasCircuitoF2[listaIndicesVarDoisDois[i][1]], '0' )
#         listaSaidasCircuitoF2.append(repP0_2)
        repP1 = saidasCircuitoF2.replace(saidasCircuitoF2[listaIndicesVarDoisDois[i][0]], '1' )
#         repP1_2 = saidasCircuitoF2.replace(saidasCircuitoF2[listaIndicesVarDoisDois[i][1]], '1' )
        listaSaidasCircuitoF2.append(repP1)
#         listaSaidasCircuitoF2.append(repP1_2)
        
    return sorted(set(listaSaidasCircuitoF2))

def simularFalhasPorta12x2():
        # Simular 10 falhas aleatórias no circuito
    listaIndicesVar = []
    listaSaidasCircuitoF1 = []
    listaIndicesVarDoisDois = []

    saidasCircuito = listCircuito[::-1] # Pegando as saídas Finais dos Circuitos
    saidasCircuitoF1 = saidasCircuito[1]
#     print(saidasCircuitoF1)
    for i in range(len(saidasCircuitoF1)): # pegando os índices das variáveis para simular falhas de entrada no circuito
        for j in range(len(var)):
            if saidasCircuitoF1[i] == var[j]:
                listaIndicesVar.append(i)

    # Pegando todas as combinações 2 a 2 e gerando 10 falhas aleatórias no circuito
    for subset in permutations(listaIndicesVar, 2):
        listaIndicesVarDoisDois.append(subset)
        
    for i in range(len(listaIndicesVarDoisDois)):
#         print(listaIndicesVarDoisDois[i][0],'--',listaIndicesVarDoisDois[i][1])
        
        repP0 = saidasCircuitoF1.replace(saidasCircuitoF1[listaIndicesVarDoisDois[i][0]], '0' )
        repP0_2 = saidasCircuitoF1.replace(saidasCircuitoF1[listaIndicesVarDoisDois[i][1]], '0' )
        listaSaidasCircuitoF1.append(repP0)
        listaSaidasCircuitoF1.append(repP0_2)

        repP1 = saidasCircuitoF1.replace(saidasCircuitoF1[listaIndicesVarDoisDois[i][0]], '1' )
        repP1_2 = saidasCircuitoF1.replace(saidasCircuitoF1[listaIndicesVarDoisDois[i][1]], '1' )
        listaSaidasCircuitoF1.append(repP1)
        listaSaidasCircuitoF1.append(repP1_2)

        repP0_1 = saidasCircuitoF1.replace(saidasCircuitoF1[listaIndicesVarDoisDois[i][0]], '1' )
        repP0_0 = saidasCircuitoF1.replace(saidasCircuitoF1[listaIndicesVarDoisDois[i][1]], '0' )
        listaSaidasCircuitoF1.append(repP0_1)
        listaSaidasCircuitoF1.append(repP0_0)

        repP1_0 = saidasCircuitoF1.replace(saidasCircuitoF1[listaIndicesVarDoisDois[i][0]], '0' )
        repP1_1 = saidasCircuitoF1.replace(saidasCircuitoF1[listaIndicesVarDoisDois[i][1]], '1' )
        listaSaidasCircuitoF1.append(repP1_0)
        listaSaidasCircuitoF1.append(repP1_1)

#         if i==10:
#             break

#     listaSaidasCircuitoF1.append(saidasCircuitoF1)
#     type((listaSaidasCircuitoF1))
    return sorted(set(listaSaidasCircuitoF1))

def simularFalhasPorta2():
        # Simular 10 falhas aleatórias no circuito
    listaIndicesVar = []
    listaSaidasCircuitoF1 = []

    saidasCircuito = listCircuito[::-1] # Pegando as saídas Finais dos Circuitos
    saidasCircuitoF1 = saidasCircuito[1]
#     print(saidasCircuitoF1)
    for i in range(len(saidasCircuitoF1)): # pegando os índices das variáveis para simular falhas de entrada no circuito
        for j in range(len(var)):
            if saidasCircuitoF1[i] == var[j]:
                listaIndicesVar.append(i)

    # Gerando 10 falhas aleatórias 1 a 1
    for i in range(len(listaIndicesVar)):    
        repP0 = saidasCircuitoF1.replace(saidasCircuitoF1[listaIndicesVar[i]], '0' )
        listaSaidasCircuitoF1.append(repP0)

    for i in range(len(listaIndicesVar)):    
        repP1 = saidasCircuitoF1.replace(saidasCircuitoF1[listaIndicesVar[i]], '1' )
        listaSaidasCircuitoF1.append(repP1)

    listaSaidasCircuitoF1.append(saidasCircuitoF1)
#     type((listaSaidasCircuitoF1))
    return sorted(set(listaSaidasCircuitoF1))

def simularFalhasPorta1():
    # Simular 10 falhas aleatórias no circuito
    listaIndicesVar = []
    listaSaidasCircuitoF2 = []

    saidasCircuito = listCircuito[::-1] # Pegando as saídas Finais dos Circuitos
    saidasCircuitoF2 = saidasCircuito[0]
#     print('saidasCircuitoF1 ', saidasCircuitoF2)
    
    for i in range(len(saidasCircuitoF2)): # pegando os índices das variáveis para simular falhas de entrada no circuito
        for j in range(len(var)):
            if saidasCircuitoF2[i] == var[j]:
                listaIndicesVar.append(i)

    # Gerando 10 falhas aleatórias 1 a 1
    for i in range(len(listaIndicesVar)):    
    #     for j in range(len(saidasCircuitoF1)): # pegando os índices das variáveis para simular falhas de entrada no circuito
        repP0 = saidasCircuitoF2.replace(saidasCircuitoF2[listaIndicesVar[i]], '0' )
        repP1 = saidasCircuitoF2.replace(saidasCircuitoF2[listaIndicesVar[i]], '1' )
        listaSaidasCircuitoF2.append(repP0)
        listaSaidasCircuitoF2.append(repP1)
        ## Adicionando no circuito as falhas 1 x 1 presa em 0 e 1
    ## Adicionando a lista de ocorrencias para gerar a TV
    for i in range(len((listaSaidasCircuitoF2))):
        print(listaSaidasCircuitoF2[i])
#         print('aqui ',saidasCircuitoF1, len(saidasCircuitoF1))

    listaSaidasCircuitoF2.append(saidasCircuitoF2)
    return sorted(set(listaSaidasCircuitoF2))

def simularFalhasPorta3():
    # Simular 10 falhas aleatórias no circuito
    listaIndicesVar = []
    listaSaidasCircuitoF3 = []

    saidasCircuito = listCircuito[::-1] # Pegando as saídas Finais dos Circuitos
    saidasCircuitoF3 = saidasCircuito[3]
    
    for i in range(len(saidasCircuitoF3)): # pegando os índices das variáveis para simular falhas de entrada no circuito
        for j in range(len(var)):
            if saidasCircuitoF3[i] == var[j]:
                listaIndicesVar.append(i)

    # Gerando 10 falhas aleatórias 1 a 1
    for i in range(len(listaIndicesVar)):    
        repP0 = saidasCircuitoF3.replace(saidasCircuitoF3[listaIndicesVar[i]], '0' )
        repP1 = saidasCircuitoF3.replace(saidasCircuitoF3[listaIndicesVar[i]], '1' )
        listaSaidasCircuitoF3.append(repP0)
        listaSaidasCircuitoF3.append(repP1)
        
    for i in range(len((listaSaidasCircuitoF3))):
        print(listaSaidasCircuitoF3[i])
#         print('aqui ',saidasCircuitoF1, len(saidasCircuitoF1))

    listaSaidasCircuitoF3.append(saidasCircuitoF3)
    return sorted(set(listaSaidasCircuitoF3))


# - <a id='#4'>4. Essa é a etapa de gerar a tabela verdade do Circuito com falha ou sem falha </a>  
###### - <a href='#8'> voltar ao sumário </a>  


Nessa etapa geramos as tabelas verdade dos circuitos com falha e sem falha, bem como as tabelas verdade das combinações 1 a 1, 2 a 2 e 3 a 3 para que possamos em uma outra etapa posterior comparar com a saída do circuito com falha.

In [895]:
### Informe o nome do arquivo para saída do Circuito    
# gerando a tabela verdade do circuito com falha ou sem falha - (O usuário escolhe qual gerar)
print( Truths(var, listCircuito, ints=True) )


+---+---+---+---+---+---+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------+-------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| A | B | C | D | E | F | ( n

### Essa é a etapa de gerar o circuito com 10 falhas aleatórias

lembrando que a análise será feita no csv da útima execução, assim recomenda que se gere o circuito sem falha e logo em seguida o circuito com falha.



In [896]:
    ## Gerando Falhas para o Circuito 2
print( TruthsGenerateF2(var, simularFalhasPorta2(), ints=True) )  # ok


+---+---+---+---+---+---+-------------------------------------------------------------------------+-------------------------------------------------------------------------+-------------------------------------------------------------------------+-------------------------------------------------------------------------+-------------------------------------------------------------------------+
| A | B | C | D | E | F |  ( not ( ( not ( 0 and  D ) ) or  ( not ( ( not ( 0 and  D ) ) ) ) ) )  |  ( not ( ( not ( 1 and  D ) ) or  ( not ( ( not ( 1 and  D ) ) ) ) ) )  |  ( not ( ( not ( C and  0 ) ) or  ( not ( ( not ( C and  0 ) ) ) ) ) )  |  ( not ( ( not ( C and  1 ) ) or  ( not ( ( not ( C and  1 ) ) ) ) ) )  |  ( not ( ( not ( C and  D ) ) or  ( not ( ( not ( C and  D ) ) ) ) ) )  |
+---+---+---+---+---+---+-------------------------------------------------------------------------+-------------------------------------------------------------------------+-----------------------------------

In [897]:
#     Gerando Falhas para o Circuito 1
print( TruthsGenerateF1(var, simularFalhasPorta1(), ints=True) )


 ( not ( ( not ( ( ( not ( 0 and  0 ) ) and  ( ( 0 and  0 ) or  ( ( D or E ) ) ) ) and  ( not ( 0 and  D ) ) ) ) and ( not ( ( not ( ( not ( 0 and  ( not ( 0 and  D ) ) ) ) and  ( not ( ( not ( 0 and  D ) ) and  E ) )  ) ) and ( ( ( 0 and ( A ^  0 ) ) or ( 0 and A ) ) ) ) ) ) )
 ( not ( ( not ( ( ( not ( 0 and  1 ) ) and  ( ( 0 and  1 ) or  ( ( D or E ) ) ) ) and  ( not ( 1 and  D ) ) ) ) and ( not ( ( not ( ( not ( 0 and  ( not ( 1 and  D ) ) ) ) and  ( not ( ( not ( 1 and  D ) ) and  E ) )  ) ) and ( ( ( 1 and ( A ^  0 ) ) or ( 0 and A ) ) ) ) ) ) )
 ( not ( ( not ( ( ( not ( 0 and  0 ) ) and  ( ( 0 and  0 ) or  ( ( D or E ) ) ) ) and  ( not ( 0 and  D ) ) ) ) and ( not ( ( not ( ( not ( 0 and  ( not ( 0 and  D ) ) ) ) and  ( not ( ( not ( 0 and  D ) ) and  E ) )  ) ) and ( ( ( 0 and ( A ^  0 ) ) or ( 0 and A ) ) ) ) ) ) )
 ( not ( ( not ( ( ( not ( 0 and  1 ) ) and  ( ( 0 and  1 ) or  ( ( D or E ) ) ) ) and  ( not ( 1 and  D ) ) ) ) and ( not ( ( not ( ( not ( 0 and  ( not ( 1 and  

In [898]:
## Gerando Falhas para o Circuito 2. 2x2
print( TruthsGenerateF2_dois_dois(var, simularFalhasPorta22x2(), ints=True) )  # ok


+---+---+---+---+---+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------

In [899]:
## Gerando Falhas para o Circuito 2. 2x2
print( TruthsGenerateF1_dois_dois(var, simularFalhasPorta12x2(), ints=True) )  # ok


+---+---+---+---+---+---+-------------------------------------------------------------------------+-------------------------------------------------------------------------+-------------------------------------------------------------------------+-------------------------------------------------------------------------+
| A | B | C | D | E | F |  ( not ( ( not ( 0 and  D ) ) or  ( not ( ( not ( 0 and  D ) ) ) ) ) )  |  ( not ( ( not ( 1 and  D ) ) or  ( not ( ( not ( 1 and  D ) ) ) ) ) )  |  ( not ( ( not ( C and  0 ) ) or  ( not ( ( not ( C and  0 ) ) ) ) ) )  |  ( not ( ( not ( C and  1 ) ) or  ( not ( ( not ( C and  1 ) ) ) ) ) )  |
+---+---+---+---+---+---+-------------------------------------------------------------------------+-------------------------------------------------------------------------+-------------------------------------------------------------------------+-------------------------------------------------------------------------+
| 0 | 0 | 0 | 0 | 0 | 0 |         

In [900]:
## Gerando Falhas para o Circuito 2. 3x3
print( TruthsGenerateF1_tres_tres(var, simularFalhasPorta13x3(), ints=True) )  # ok


+---+---+---+---+---+---+-------------------------------------------------------------------------+-------------------------------------------------------------------------+-------------------------------------------------------------------------+
| A | B | C | D | E | F |  ( not ( ( not ( 0 and  D ) ) or  ( not ( ( not ( 0 and  D ) ) ) ) ) )  |  ( not ( ( not ( 1 and  D ) ) or  ( not ( ( not ( 1 and  D ) ) ) ) ) )  |  ( not ( ( not ( C and  1 ) ) or  ( not ( ( not ( C and  1 ) ) ) ) ) )  |
+---+---+---+---+---+---+-------------------------------------------------------------------------+-------------------------------------------------------------------------+-------------------------------------------------------------------------+
| 0 | 0 | 0 | 0 | 0 | 0 |                                    0                                    |                                    0                                    |                                    0                                    |
| 0 | 0 

In [901]:
## Gerando Falhas para o Circuito 2. 3x3
print( TruthsGenerateF2_tres_tres(var, simularFalhasPorta23x3(), ints=True) )  # ok


+---+---+---+---+---+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------

In [902]:
## Gerando Falhas para o Circuito 2. 3x3
print( TruthsGenerateF3(var, simularFalhasPorta3(), ints=True) )  
# Se for o circuito final descomente essa linha para pegar a 3° final porta do circuito
 

( not ( ( not ( ( not ( 0 and  ( not ( 0 and  D ) ) ) ) and  ( not ( ( not ( 0 and  D ) ) and  E ) )  ) ) and ( ( ( 0 and ( A ^  0 ) ) or ( 0 and A ) ) ) ) )
( not ( ( not ( ( not ( 0 and  ( not ( 1 and  D ) ) ) ) and  ( not ( ( not ( 1 and  D ) ) and  E ) )  ) ) and ( ( ( 1 and ( A ^  0 ) ) or ( 0 and A ) ) ) ) )
( not ( ( not ( ( not ( 0 and  ( not ( C and  0 ) ) ) ) and  ( not ( ( not ( C and  0 ) ) and  E ) )  ) ) and ( ( ( C and ( A ^  0 ) ) or ( 0 and A ) ) ) ) )
( not ( ( not ( ( not ( 0 and  ( not ( C and  1 ) ) ) ) and  ( not ( ( not ( C and  1 ) ) and  E ) )  ) ) and ( ( ( C and ( A ^  0 ) ) or ( 0 and A ) ) ) ) )
( not ( ( not ( ( not ( 0 and  ( not ( 0 and  D ) ) ) ) and  ( not ( ( not ( 0 and  D ) ) and  E ) )  ) ) and ( ( ( 0 and ( A ^  0 ) ) or ( 0 and A ) ) ) ) )
( not ( ( not ( ( not ( 0 and  ( not ( 1 and  D ) ) ) ) and  ( not ( ( not ( 1 and  D ) ) and  E ) )  ) ) and ( ( ( 1 and ( A ^  0 ) ) or ( 0 and A ) ) ) ) )
( not ( ( not ( ( not ( 0 and  ( not ( C and  0 ) ) 

# - <a id='#4.1'>4.1 Leitura dos csvs gerados. Com falha, sem falha e os csvs das saídas dos circuitos. </a>  
###### - <a href='#8'> voltar ao sumário </a>  


# Abaixo é lido os csv com falha e sem falha'

In [903]:
dfFilesGeradoFalha = pd.read_csv('saidaCircuitoCSV1Falha.csv')

dfFilesGeradoSemFalha = pd.read_csv('saidaCircuitoCSV1SemFalha.csv')

## Salvando o arquivo com falhas parciais e fazendo o merge em um novo arquivo
dfFilesFalhasS1 = pd.read_csv('SaidasFalhasS1.csv')
# print(dfFilesFalhasS1.head(8) )
dfFilesFalhasS2 = pd.read_csv('SaidasFalhasS2.csv')
# print(dfFilesFalhasS2.head(8) )
dfFilesFalhasS1_2x2 = pd.read_csv('SaidasFalhasS2_2x2.csv')
dfFilesFalhasS2_2x2 = pd.read_csv('SaidasFalhasS1_2x2.csv')
dfFilesFalhasS1_3x3 = pd.read_csv('SaidasFalhasS2_3x3.csv')
dfFilesFalhasS2_3x3 = pd.read_csv('SaidasFalhasS1_3x3.csv')

dfFilesFalhasS3 = pd.read_csv('SaidasFalhasS3.csv')  # Se for o circuito final descomente essa linha



### A seguir observamos a saída dos circuitos gerados com falha, sem falha e das combinações de falhas geradas

In [904]:
# caso já tenha gerado o csv com falha veja aqui a tabela verdade
dfFilesGeradoFalha.head(8)

,A,B,C,D,E,F,( not ( C and D ) ),( not ( ( not ( C and D ) ) ) ),( not ( 1 and ( not ( C and D ) ) ) ),( not ( D and E ) ),( not ( ( not ( C and D ) ) or ( not ( ( not ( C and D ) ) ) ) ) ),( not ( ( not ( 1 and ( not ( C and D ) ) ) ) and ( not ( D and E ) ) ) )
0,0,0,0,0,0,0,1,0,0,1,0,1
1,0,0,0,0,0,1,1,0,0,1,0,1
2,0,0,0,0,1,0,1,0,0,1,0,1
3,0,0,0,0,1,1,1,0,0,1,0,1
4,0,0,0,1,0,0,1,0,0,1,0,1
5,0,0,0,1,0,1,1,0,0,1,0,1
6,0,0,0,1,1,0,1,0,0,0,0,1
7,0,0,0,1,1,1,1,0,0,0,0,1


In [905]:
# caso já tenha gerado o csv sem falha veja aqui a tabela verdade
dfFilesGeradoSemFalha.head(8)   

,A,B,C,D,E,F,( not ( ( not ( ( not ( A and C ) ) and ( not ( 0 and ( not ( C and D ) ) ) ) ) ) and ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) ) ),( not ( ( not ( ( not ( A and C ) ) and ( not ( 0 and ( not ( C and D ) ) ) ) ) ) and D ) ),( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ),( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ),( ( ( ( A ^ 0 ) ^ C ) ) ),( not ( ( not ( C and D ) ) or ( not ( ( not ( C and D ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) )
0,0,0,0,0,0,0,1,1,1,1,0,0,0
1,0,0,0,0,0,1,1,1,1,1,0,0,0
2,0,0,0,0,1,0,1,1,0,1,0,0,1
3,0,0,0,0,1,1,1,1,0,1,0,0,1
4,0,0,0,1,0,0,1,1,0,1,0,0,1
5,0,0,0,1,0,1,1,1,0,1,0,0,1
6,0,0,0,1,1,0,1,1,0,1,0,0,1
7,0,0,0,1,1,1,1,1,0,1,0,0,1


### Observaremos agora as falhas para cada saída do circuito 
Cada coluna representa uma saída gerada e possível solução do circuito



In [906]:
dfFilesFalhasS1.head(8)  # Porta de Saída 1 do circuito

,A,B,C,D,E,F,( not ( ( not ( ( ( not ( 0 and 0 ) ) and ( ( 0 and 0 ) or ( ( D or E ) ) ) ) and ( not ( 0 and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( 0 and D ) ) ) ) and ( not ( ( not ( 0 and D ) ) and E ) ) ) ) and ( ( ( 0 and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and 1 ) ) and ( ( 0 and 1 ) or ( ( D or E ) ) ) ) and ( not ( 1 and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( 1 and D ) ) ) ) and ( not ( ( not ( 1 and D ) ) and E ) ) ) ) and ( ( ( 1 and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( 0 or E ) ) ) ) and ( not ( C and 0 ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and 0 ) ) ) ) and ( not ( ( not ( C and 0 ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( 1 or E ) ) ) ) and ( not ( C and 1 ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and 1 ) ) ) ) and ( not ( ( not ( C and 1 ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or 0 ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and 0 ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or 1 ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and 1 ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( 0 ^ 0 ) ) or ( 0 and 0 ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( 1 ^ 0 ) ) or ( 0 and 1 ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) )
0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0
2,0,0,0,0,1,0,1,1,1,1,0,1,1,1,1
3,0,0,0,0,1,1,1,1,1,1,0,1,1,1,1
4,0,0,0,1,0,0,1,0,0,1,1,1,1,1,1
5,0,0,0,1,0,1,1,0,0,1,1,1,1,1,1
6,0,0,0,1,1,0,1,0,1,1,1,1,1,1,1
7,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1


In [907]:
dfFilesFalhasS2.head(8) # Porta de Saída 2 do circuito

,A,B,C,D,E,F,( not ( ( not ( 0 and D ) ) or ( not ( ( not ( 0 and D ) ) ) ) ) ),( not ( ( not ( 1 and D ) ) or ( not ( ( not ( 1 and D ) ) ) ) ) ),( not ( ( not ( C and 0 ) ) or ( not ( ( not ( C and 0 ) ) ) ) ) ),( not ( ( not ( C and 1 ) ) or ( not ( ( not ( C and 1 ) ) ) ) ) ),( not ( ( not ( C and D ) ) or ( not ( ( not ( C and D ) ) ) ) ) )
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0
5,0,0,0,1,0,1,0,0,0,0,0
6,0,0,0,1,1,0,0,0,0,0,0
7,0,0,0,1,1,1,0,0,0,0,0


In [908]:
dfFilesFalhasS1_2x2.head(8) # Combinação 2x2 do circuito. Saída 1 do circuito

,A,B,C,D,E,F,( not ( ( not ( ( ( not ( 0 and 0 ) ) and ( ( 0 and 0 ) or ( ( D or E ) ) ) ) and ( not ( 0 and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( 0 and D ) ) ) ) and ( not ( ( not ( 0 and D ) ) and E ) ) ) ) and ( ( ( 0 and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and 1 ) ) and ( ( 0 and 1 ) or ( ( D or E ) ) ) ) and ( not ( 1 and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( 1 and D ) ) ) ) and ( not ( ( not ( 1 and D ) ) and E ) ) ) ) and ( ( ( 1 and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( 0 or E ) ) ) ) and ( not ( C and 0 ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and 0 ) ) ) ) and ( not ( ( not ( C and 0 ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( 1 or E ) ) ) ) and ( not ( C and 1 ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and 1 ) ) ) ) and ( not ( ( not ( C and 1 ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or 0 ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and 0 ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or 1 ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and 1 ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( 0 ^ 0 ) ) or ( 0 and 0 ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( 1 ^ 0 ) ) or ( 0 and 1 ) ) ) ) ) ) )
0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1,0,0,0,0,0,1,0,0,0,1,0,1,0,0
2,0,0,0,0,1,0,1,1,1,1,0,1,1,1
3,0,0,0,0,1,1,1,1,1,1,0,1,1,1
4,0,0,0,1,0,0,1,0,0,1,1,1,1,1
5,0,0,0,1,0,1,1,0,0,1,1,1,1,1
6,0,0,0,1,1,0,1,0,1,1,1,1,1,1
7,0,0,0,1,1,1,1,0,1,1,1,1,1,1


In [909]:
dfFilesFalhasS2_2x2.head(8) # Combinação 2x2 do circuito. Saída 2 do circuito

,A,B,C,D,E,F,( not ( ( not ( 0 and D ) ) or ( not ( ( not ( 0 and D ) ) ) ) ) ),( not ( ( not ( 1 and D ) ) or ( not ( ( not ( 1 and D ) ) ) ) ) ),( not ( ( not ( C and 0 ) ) or ( not ( ( not ( C and 0 ) ) ) ) ) ),( not ( ( not ( C and 1 ) ) or ( not ( ( not ( C and 1 ) ) ) ) ) )
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,1,1,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0
5,0,0,0,1,0,1,0,0,0,0
6,0,0,0,1,1,0,0,0,0,0
7,0,0,0,1,1,1,0,0,0,0


In [910]:
dfFilesFalhasS1_3x3.head(8) # Combinação 3x3 do circuito. Saída 1 do circuito

,A,B,C,D,E,F,( not ( ( not ( ( ( not ( 0 and 0 ) ) and ( ( 0 and 0 ) or ( ( D or E ) ) ) ) and ( not ( 0 and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( 0 and D ) ) ) ) and ( not ( ( not ( 0 and D ) ) and E ) ) ) ) and ( ( ( 0 and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and 1 ) ) and ( ( 0 and 1 ) or ( ( D or E ) ) ) ) and ( not ( 1 and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( 1 and D ) ) ) ) and ( not ( ( not ( 1 and D ) ) and E ) ) ) ) and ( ( ( 1 and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( 1 or E ) ) ) ) and ( not ( C and 1 ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and 1 ) ) ) ) and ( not ( ( not ( C and 1 ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or 1 ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and 1 ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( 1 ^ 0 ) ) or ( 0 and 1 ) ) ) ) ) ) )
0,0,0,0,0,0,0,0,0,1,1,0
1,0,0,0,0,0,1,0,0,1,1,0
2,0,0,0,0,1,0,1,1,1,1,1
3,0,0,0,0,1,1,1,1,1,1,1
4,0,0,0,1,0,0,1,0,1,1,1
5,0,0,0,1,0,1,1,0,1,1,1
6,0,0,0,1,1,0,1,0,1,1,1
7,0,0,0,1,1,1,1,0,1,1,1


In [911]:
dfFilesFalhasS2_3x3.head(8) # Combinação 3x3 do circuito. Saída 2 do circuito

,A,B,C,D,E,F,( not ( ( not ( 0 and D ) ) or ( not ( ( not ( 0 and D ) ) ) ) ) ),( not ( ( not ( 1 and D ) ) or ( not ( ( not ( 1 and D ) ) ) ) ) ),( not ( ( not ( C and 1 ) ) or ( not ( ( not ( C and 1 ) ) ) ) ) )
0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,1,0,0,0
4,0,0,0,1,0,0,0,0,0
5,0,0,0,1,0,1,0,0,0
6,0,0,0,1,1,0,0,0,0
7,0,0,0,1,1,1,0,0,0


In [912]:
dfFilesFalhasS3.head(8) #  SE FOR O CIRCUITO FINAL descomente essa linha

,A,B,C,D,E,F,( not ( ( not ( ( not ( 0 and ( not ( 0 and D ) ) ) ) and ( not ( ( not ( 0 and D ) ) and E ) ) ) ) and ( ( ( 0 and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ),( not ( ( not ( ( not ( 0 and ( not ( 1 and D ) ) ) ) and ( not ( ( not ( 1 and D ) ) and E ) ) ) ) and ( ( ( 1 and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ),( not ( ( not ( ( not ( 0 and ( not ( C and 0 ) ) ) ) and ( not ( ( not ( C and 0 ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ),( not ( ( not ( ( not ( 0 and ( not ( C and 1 ) ) ) ) and ( not ( ( not ( C and 1 ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ),( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and 0 ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ),( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and 1 ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ),( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( 0 ^ 0 ) ) or ( 0 and 0 ) ) ) ) ),( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( 1 ^ 0 ) ) or ( 0 and 1 ) ) ) ) ),( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) )
0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1
1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1
2,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1
3,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1
4,0,0,0,1,0,0,1,1,1,1,1,1,1,1,1
5,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1
6,0,0,0,1,1,0,1,1,1,1,1,1,1,1,1
7,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1


In [913]:
data = pd.read_csv(saidaCircuitoCsv+'.csv')
data.head(8)  # vendo os primeiros valores da tabela verdade

,A,B,C,D,E,F,( not ( ( not ( ( not ( A and C ) ) and ( not ( 0 and ( not ( C and D ) ) ) ) ) ) and ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) ) ),( not ( ( not ( ( not ( A and C ) ) and ( not ( 0 and ( not ( C and D ) ) ) ) ) ) and D ) ),( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ),( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ),( ( ( ( A ^ 0 ) ^ C ) ) ),( not ( ( not ( C and D ) ) or ( not ( ( not ( C and D ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) )
0,0,0,0,0,0,0,1,1,1,1,0,0,0
1,0,0,0,0,0,1,1,1,1,1,0,0,0
2,0,0,0,0,1,0,1,1,0,1,0,0,1
3,0,0,0,0,1,1,1,1,0,1,0,0,1
4,0,0,0,1,0,0,1,1,0,1,0,0,1
5,0,0,0,1,0,1,1,1,0,1,0,0,1
6,0,0,0,1,1,0,1,1,0,1,0,0,1
7,0,0,0,1,1,1,1,1,0,1,0,0,1


## Salvando o circuito lógico paranteseado
 
Uma forma de guardar o circuito lógico.

In [914]:

mapCircuito = dict(zip(listaPortas, listCircuito))
print("Lista: ",mapCircuito)
#  Escrevendo a forma lógica do circuito paranteseada no formato de entrada
logicForm = open('logicFormCircuit1_Final.txt','w')
logicForm.write("var = " + str(var))
logicForm.write("\n")
logicForm.write(str(mapCircuito))
logicForm.close()

Lista:  {'p1 ': '( not ( ( not ( ( not ( A and  C ) ) and  ( not ( 0 and  ( not ( C and  D ) ) ) ) ) ) and  ( not ( ( not ( 0 and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) ) )', 'p2 ': '( not ( ( not ( ( not ( A and  C ) ) and  ( not ( 0 and  ( not ( C and  D ) ) ) ) ) ) and  D ) )', 'p3 ': '( not ( ( ( not ( 0 and  C ) ) and  ( ( 0 and  C ) or  ( ( D or E ) ) ) ) and  ( not ( C and  D ) ) ) )', 'p4 ': '( not ( ( not ( ( not ( 0 and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) and ( ( ( C and ( A ^  0 ) ) or ( 0 and A ) ) ) ) )', 'pF1': ' ( ( ( ( A ^  0 ) ^ C ) ) )', 'pF2': ' ( not ( ( not ( C and  D ) ) or  ( not ( ( not ( C and  D ) ) ) ) ) ) ', 'pF3': ' ( not ( ( not ( ( ( not ( 0 and  C ) ) and  ( ( 0 and  C ) or  ( ( D or E ) ) ) ) and  ( not ( C and  D ) ) ) ) and ( not ( ( not ( ( not ( 0 and  ( not ( C and  D ) ) ) ) and  ( not ( ( not ( C and  D ) ) and  E ) )  ) ) and ( ( ( C and ( A ^  0 ) ) or ( 0 and A ) ) ) 

### Guardando a tabela do circuito com falha em um arquivo de texto

In [915]:
#  Escrevendo a forma lógica do circuito paranteseada
logicForm = open('TVCircuit1.txt','w')
logicForm.write(str( (str(listCircuito),str(data.values)  ) ))
logicForm.close()

## Lendo tabelas verdades geradas para detectar quais configurações geram a tabela verdade com Falhas


Nossa abordagem gera combinações 1x1, 2x2, 3x3 para um detemrinado circuito. Partimos da hipótese de que o circuito não poderá ter mais de duas saídas. Caso tenha apenas 2 saídas do circuito será mapeada.

In [916]:
dataComFalha = pd.read_csv('SaidasFalhasS1.csv')
dataSemFalha = pd.read_csv('SaidasFalhasS2.csv')

In [917]:
# dataSemFalha[['( ( C and ( A ^ B ) )or ( B and A ))', '( ( ( C and ( A ^ B ) )or ( B and A )))']] =0
# dataSemFalha.to_csv('saidaCircuitoCSV1Falha.csv', index=False)

In [918]:
dataSemFalha.head(8)  # Não testamos sem falha ainda. Estamos simulando as falhas diretamente.

,A,B,C,D,E,F,( not ( ( not ( 0 and D ) ) or ( not ( ( not ( 0 and D ) ) ) ) ) ),( not ( ( not ( 1 and D ) ) or ( not ( ( not ( 1 and D ) ) ) ) ) ),( not ( ( not ( C and 0 ) ) or ( not ( ( not ( C and 0 ) ) ) ) ) ),( not ( ( not ( C and 1 ) ) or ( not ( ( not ( C and 1 ) ) ) ) ) ),( not ( ( not ( C and D ) ) or ( not ( ( not ( C and D ) ) ) ) ) )
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0
5,0,0,0,1,0,1,0,0,0,0,0
6,0,0,0,1,1,0,0,0,0,0,0
7,0,0,0,1,1,1,0,0,0,0,0


In [919]:
dataComFalha.head(8)

,A,B,C,D,E,F,( not ( ( not ( ( ( not ( 0 and 0 ) ) and ( ( 0 and 0 ) or ( ( D or E ) ) ) ) and ( not ( 0 and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( 0 and D ) ) ) ) and ( not ( ( not ( 0 and D ) ) and E ) ) ) ) and ( ( ( 0 and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and 1 ) ) and ( ( 0 and 1 ) or ( ( D or E ) ) ) ) and ( not ( 1 and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( 1 and D ) ) ) ) and ( not ( ( not ( 1 and D ) ) and E ) ) ) ) and ( ( ( 1 and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( 0 or E ) ) ) ) and ( not ( C and 0 ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and 0 ) ) ) ) and ( not ( ( not ( C and 0 ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( 1 or E ) ) ) ) and ( not ( C and 1 ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and 1 ) ) ) ) and ( not ( ( not ( C and 1 ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or 0 ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and 0 ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or 1 ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and 1 ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( 0 ^ 0 ) ) or ( 0 and 0 ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( 1 ^ 0 ) ) or ( 0 and 1 ) ) ) ) ) ) ),( not ( ( not ( ( ( not ( 0 and C ) ) and ( ( 0 and C ) or ( ( D or E ) ) ) ) and ( not ( C and D ) ) ) ) and ( not ( ( not ( ( not ( 0 and ( not ( C and D ) ) ) ) and ( not ( ( not ( C and D ) ) and E ) ) ) ) and ( ( ( C and ( A ^ 0 ) ) or ( 0 and A ) ) ) ) ) ) )
0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0
2,0,0,0,0,1,0,1,1,1,1,0,1,1,1,1
3,0,0,0,0,1,1,1,1,1,1,0,1,1,1,1
4,0,0,0,1,0,0,1,0,0,1,1,1,1,1,1
5,0,0,0,1,0,1,1,0,0,1,1,1,1,1,1
6,0,0,0,1,1,0,1,0,1,1,1,1,1,1,1
7,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1


# - <a id='#5'>5 Comparando as falhas com o circuito de erro e observando quais falhas são iguais as saidas do circuito</a>  
###### - <a href='#8'> voltar ao sumário </a>  


##### Essa é uma etapa de diagnóstico, assim que verificarmos para cada saída do circuito se vai parte da solução do circuito, ou seja, se é igual a saída do circuito com falha informado.

In [920]:
# falha 1x1
indicesFalhasIguaisF1 = [] 
indicesFalhasIguaisF2 = [] 

for i, row in dfFilesFalhasS1.iterrows():    
    indicesFalhasIguaisF1.append(dfFilesGeradoFalha[dfFilesGeradoFalha.columns[::-1][6]].equals(dfFilesFalhasS1[dfFilesFalhasS1.columns[row][0]]))

for i, row in dfFilesFalhasS2.iterrows():    
    indicesFalhasIguaisF2.append(dfFilesGeradoFalha[dfFilesGeradoFalha.columns[::-1][5]].equals(dfFilesFalhasS2[dfFilesFalhasS2.columns[row][0]]))
    

In [921]:
# falha 2x2
indicesFalhasIguaisF12x2 = [] 
indicesFalhasIguaisF22x2 = [] 

for i, row in dfFilesFalhasS1.iterrows():    
    indicesFalhasIguaisF12x2.append(dfFilesGeradoFalha[dfFilesGeradoFalha.columns[::-1][6]].equals(dfFilesFalhasS1_2x2[dfFilesFalhasS1_2x2.columns[row][0]]))

for i, row in dfFilesFalhasS2.iterrows():    
    indicesFalhasIguaisF22x2.append(dfFilesGeradoFalha[dfFilesGeradoFalha.columns[::-1][5]].equals(dfFilesFalhasS2_2x2[dfFilesFalhasS2_2x2.columns[row][0]]))


In [922]:
# falha 3x3
indicesFalhasIguaisF13x3 = [] 
indicesFalhasIguaisF23x3 = [] 

for i, row in dfFilesFalhasS1.iterrows():    
    indicesFalhasIguaisF13x3.append(dfFilesGeradoFalha[dfFilesGeradoFalha.columns[::-1][6]].equals(dfFilesFalhasS1_3x3[dfFilesFalhasS1_3x3.columns[row][0]]))

for i, row in dfFilesFalhasS2.iterrows():    
    indicesFalhasIguaisF23x3.append(dfFilesGeradoFalha[dfFilesGeradoFalha.columns[::-1][5]].equals(dfFilesFalhasS2_3x3[dfFilesFalhasS2_3x3.columns[row][0]]))


In [923]:
# Pegando os índices da lista para gerar as colunas que geram a falha do circuito 
indicesFalhasIguaisF1_ = [] 
indicesFalhasIguaisF2_ = [] 

for i in range(len(indicesFalhasIguaisF1)):
    if indicesFalhasIguaisF1[i]==True:
        indicesFalhasIguaisF1_.append(i+3) # i+3 por que precisamos filtrar apenas as colunas que são iguais
#         print(i)
for i in range(len(indicesFalhasIguaisF2)):
    if indicesFalhasIguaisF2[i]==True:
        indicesFalhasIguaisF2_.append(i+3)
#         print(i)
    

In [924]:
# Pegando os índices da lista para gerar as colunas que geram a falha do circuito 
indicesFalhasIguaisF1_2x2 = [] 
indicesFalhasIguaisF2_2x2 = [] 

for i in range(len(indicesFalhasIguaisF12x2)):
    if indicesFalhasIguaisF12x2[i]==True:
        indicesFalhasIguaisF1_2x2.append(i+3) # i+3 por que precisamos filtrar apenas as colunas que são iguais
#         print(i)
for i in range(len(indicesFalhasIguaisF22x2)):
    if indicesFalhasIguaisF22x2[i]==True:
        indicesFalhasIguaisF2_2x2.append(i+3)
#         print(i)
    

In [925]:
# Pegando os índices da lista para gerar as colunas que geram a falha do circuito 
indicesFalhasIguaisF1_3x3 = [] 
indicesFalhasIguaisF2_3x3 = [] 

for i in range(len(indicesFalhasIguaisF13x3)):
    if indicesFalhasIguaisF13x3[i]==True:
        indicesFalhasIguaisF1_3x3.append(i+3) # i+3 por que precisamos filtrar apenas as colunas que são iguais
#         print(i)
for i in range(len(indicesFalhasIguaisF23x3)):
    if indicesFalhasIguaisF23x3[i]==True:
        indicesFalhasIguaisF2_3x3.append(i+3)
#         print(i)
    

In [926]:
# # falha 1x1 # Se for o circuito final descomente isso
indicesFalhasIguaisF3 = [] 
indicesFalhasIguaisF3_ = [] 

for i, row in dfFilesFalhasS3.iterrows():    
    indicesFalhasIguaisF3.append(dfFilesGeradoFalha[dfFilesGeradoFalha.columns[::-1][6]].equals(dfFilesFalhasS3[dfFilesFalhasS3.columns[row][0]]))
# print(indicesFalhasIguaisF3)
#         print(i)
for i in range(len(indicesFalhasIguaisF3)):
    if indicesFalhasIguaisF3[i]==True:
        indicesFalhasIguaisF3_.append(i+3)

# - <a id='#5.1'>5.1 Resultado do diagnóstico</a>  
###### - <a href='#8'> voltar ao sumário </a>  


# Etapa do diagnostico, verificando quantas das saídas geradas são iguais a falha do circuito com falha

Caso exista uma solução gerada que seja igual a tabela verdade do circuito informado é mostrado e é contabilizado. 



In [927]:
print(len(indicesFalhasIguaisF1_), " formas de se chegar a tabela verdade do circuito com falha 1x1 combinações")

0  formas de se chegar a tabela verdade do circuito com falha 1x1 combinações


In [928]:
## Filtrando quais soluções geram o circuito informado.
dfFilesFalhasS1[dfFilesFalhasS1.columns[indicesFalhasIguaisF1_]] 

""
0
1
2
3
4
5
6
7
8
9


In [929]:
print(len(indicesFalhasIguaisF2_), " formas de se chegar a tabela verdade do circuito com falha")

0  formas de se chegar a tabela verdade do circuito com falha


In [930]:
dfFilesFalhasS2[dfFilesFalhasS2.columns[indicesFalhasIguaisF2_]] 

""
0
1
2
3
4
5
6
7
8
9


In [931]:
print(len(indicesFalhasIguaisF1), " formas de se chegar a tabela verdade do circuito com falha")
print(len(indicesFalhasIguaisF12x2), " formas de se chegar a tabela verdade do circuito com falha")
print(len(indicesFalhasIguaisF13x3), " formas de se chegar a tabela verdade do circuito com falha")

64  formas de se chegar a tabela verdade do circuito com falha
64  formas de se chegar a tabela verdade do circuito com falha
64  formas de se chegar a tabela verdade do circuito com falha


In [932]:
# Se for o circuito final descomente isso
# print(len(indicesFalhasIguaisF3_), " formas de se chegar a tabela verdade do circuito com falha")
# dfFilesFalhasS3[dfFilesFalhasS3.columns[indicesFalhasIguaisF3_]] 

# - <a id='#6'>6. Abaixo salvamos as saídas do circuito, fazemos um diagnóstico</a>  
###### - <a href='#8'> voltar ao sumário </a>  


In [933]:
# Diagnostico do Circuito  Escrevendo os valores pelo qual gera o circuito com falha.
diagTxt = open('DiagnosticoCircuito.txt','a') # O 'a' significa que o arquivo pode ser gerado, atualizado.

diagTxt.write(str("Diagnostico para o circuito ") )
diagTxt.write(str('\n'))
diagTxt.write(str(listCircuito[::-1][0]) )
diagTxt.write(str('\n'))
diagTxt.write(str('\n'))

diagTxt.write(str(dfFilesFalhasS1[dfFilesFalhasS1.columns[indicesFalhasIguaisF1_]].columns ))
diagTxt.write(str('\n'))
diagTxt.write(str('\n'))

diagTxt.write(str("Diagnostico para o circuito ") )
diagTxt.write(str('\n'))
diagTxt.write(str(listCircuito[::-1][1]) )
diagTxt.write(str('\n'))
diagTxt.write(str('\n'))

diagTxt.write(str('\n'))
diagTxt.write(str('\n'))

diagTxt.write(str(dfFilesFalhasS2[dfFilesFalhasS2.columns[indicesFalhasIguaisF2_]].columns ))
diagTxt.write(str('\n'))
diagTxt.write(str('\n'))

# # Se for o circuito final descomente isso
# diagTxt.write(str("Diagnostico para o circuito ") )
# diagTxt.write(str('\n'))
# diagTxt.write(str(listCircuito[::-1][2]) )
# diagTxt.write(str('\n'))
# diagTxt.write(str('\n'))

# diagTxt.write(str('\n'))
# diagTxt.write(str('\n'))

# diagTxt.write(str(dfFilesFalhasS3[dfFilesFalhasS3.columns[indicesFalhasIguaisF3_]].columns ))
# diagTxt.write(str('\n'))
# diagTxt.write(str('\n'))


diagTxt.write(str("Tempo de Execução do programa: "+ str(time.time() - time_diag)) )
diagTxt.close()


In [934]:
dfFilesFalhasS1_2x2[dfFilesFalhasS1_2x2.columns[indicesFalhasIguaisF1_2x2]] 

""
0
1
2
3
4
5
6
7
8
9


In [935]:
dfFilesFalhasS2_2x2[dfFilesFalhasS2_2x2.columns[indicesFalhasIguaisF2_2x2]] 

""
0
1
2
3
4
5
6
7
8
9


In [936]:

dfFilesFalhasS1_3x3[dfFilesFalhasS1_3x3.columns[indicesFalhasIguaisF1_3x3]] 

""
0
1
2
3
4
5
6
7
8
9


In [937]:
dfFilesFalhasS2_3x3[dfFilesFalhasS2_3x3.columns[indicesFalhasIguaisF2_3x3]] 

""
0
1
2
3
4
5
6
7
8
9


# - <a id='#7'> 7. Tempo de Execução do Diagnostico</a>  
###### - <a href='#8'> voltar ao sumário </a>  


In [938]:
print ("Tempo de Execução do programa: "+ str(time.time() - time_diag), " segundos")

Tempo de Execução do programa: 12.742963314056396  segundos
